## Resgatando o valor da linha de baixo *(Mesclar tabela com ela mesma)*

Muitas vezes precisamos comparar o valor da linha com o valor da linha de baixo, e no exemplo abaixo, é uma tabela de ID de Chamado, onde eu comparo com a data da linha de baixo, para saber a data final

1) Crie uma coluna Indice que começa com número 1
2) Depois crie uma coluna co o número 0
3) Mescle a coluna com ela mesmo
4) Nesse exemplo, é importante está agrupado por ID então deve mesclar por indice & ID Chamado e mescle com indice_1 & ID Chamado


Veja o Código fonte abaixo


In [ ]:
let
    #Fonte = Table.FromRows(Json.Document(Binary.Decompress(Binary.FromText("dZCxDoIwFEV/hXQmgVf6QLu5kLg56EQYMHQg0ZKgLv6N3+KPiSilDx5bk3vae3qLQoAIxe5sunvbH6SMII4kBhsNSpThP7Z1dTWWElufOBhb94AJjvsvk3BMbqtL86zq1icgHRHJegBoyKacEwGpIXbIigmFVlV+SMKrKA3JlLMq6D/hVE63x/vVNbSNoERILRDFCrl11dxnsb9aLsO3EBEckFjiECORcP/Ixus4txgb0Cfyxm9ICVJ+AA==", BinaryEncoding.Base64), Compression.Deflate)), let _t = ((type nullable text) meta [Serialized.Text = true]) in type table [#"ID Chamado" = _t, Status = _t, #"Data Inicial" = _t]),
    #"Tipo Alterado" = Table.TransformColumnTypes(Fonte,{{"ID Chamado", Int64.Type}, {"Status", type text}, {"Data Inicial", type datetime}}),

    //==================================================================================================
    // ===== "Aqui é a Etapa que eu Crio as Colunas Índices e depois eu mesclo com ela mesmo" =====
    #"Índice Adicionado" = Table.AddIndexColumn(#"Tipo Alterado", "Índice", 0, 1, Int64.Type),
    #"Índice Adicionado1" = Table.AddIndexColumn(#"Índice Adicionado", "Índice.1", 1, 1, Int64.Type),
    #"Consultas Mescladas" = Table.NestedJoin(#"Índice Adicionado1", {"Índice.1", "ID Chamado"}, #"Índice Adicionado1", {"Índice", "ID Chamado"}, "Índice Adicionado1", JoinKind.LeftOuter),
    #"Índice Adicionado1 Expandido" = Table.ExpandTableColumn(#"Consultas Mescladas", "Índice Adicionado1", {"Data Inicial"}, {"Índice Adicionado1.Data Inicial"}),
    //==================================================================================================

    #"Linhas Classificadas" = Table.Sort(#"Índice Adicionado1 Expandido",{{"ID Chamado", Order.Ascending}, {"Data Inicial", Order.Ascending}}),
    #"Colunas Removidas" = Table.RemoveColumns(#"Linhas Classificadas",{"Índice", "Índice.1"}),
    #"Colunas Renomeadas" = Table.RenameColumns(#"Colunas Removidas",{{"Índice Adicionado1.Data Inicial", "Data Final"}})
in
    #"Colunas Renomeadas"

No Exemplo abaixo, vamos comparar o valor total do dia posterior

Nesse caso primeiro teremos que fazer um agrupamento por data, com o total da coluna vamor, e a mesclagem será feira apenas por data

In [ ]:
let
    Fonte = Table.FromRows(Json.Document(Binary.Decompress(Binary.FromText("bZJhCsUgDIPv4u/BamttPcvY/a/xZLqHnYEx2EdqmszrSpnO/jCxpqN/ULqPD2QFUBDM/f3AvEIxAEsDsGYATQB0QnAa8QpbQWeiPbWC8TeRoESCEglKVFAiDUZTWdfmDUBBMJcKoPqAFtxndR725AdSbGlUR6EQH0YUYraRiNaYPO8SaYC2G7HobsRsu9FfGYzYgdGrDH3y+EdkSBnHaR9vvk/Xrrt/", BinaryEncoding.Base64), Compression.Deflate)), let _t = ((type nullable text) meta [Serialized.Text = true]) in type table [Data = _t, #"Valor Venda" = _t]),
    #"Tipo Alterado" = Table.TransformColumnTypes(Fonte,{{"Data", type date}, {"Valor Venda", Int64.Type}}),
    #"Linhas Agrupadas" = Table.Group(#"Tipo Alterado", {"Data"}, {{"Total", each List.Sum([Valor Venda]), type nullable number}}),
    #"Linhas Classificadas" = Table.Sort(#"Linhas Agrupadas",{{"Data", Order.Ascending}}),
    #"Índice Adicionado" = Table.AddIndexColumn(#"Linhas Classificadas", "Índice", 1, 1, Int64.Type),
    #"Índice Adicionado1" = Table.AddIndexColumn(#"Índice Adicionado", "Índice.1", 0, 1, Int64.Type),
    #"Consultas Mescladas" = Table.NestedJoin(#"Índice Adicionado1", {"Índice"}, #"Índice Adicionado1", {"Índice.1"}, "Índice Adicionado1", JoinKind.LeftOuter),
    #"Índice Adicionado1 Expandido" = Table.ExpandTableColumn(#"Consultas Mescladas", "Índice Adicionado1", {"Total"}, {"Total.1"}),
    #"Colunas Removidas" = Table.RemoveColumns(#"Índice Adicionado1 Expandido",{"Índice", "Índice.1"}),
    #"Personalização Adicionada" = Table.AddColumn(#"Colunas Removidas", "Valor Delta", each [Total.1] - [Total]),
    #"Colunas Removidas1" = Table.RemoveColumns(#"Personalização Adicionada",{"Total", "Total.1"})
in
    #"Colunas Removidas1"